# 전처리

In [ ]:
import pandas as pd

In [ ]:
df_review = pd.read_csv('revised_review0.csv', encoding='utf-8-sig')
df_list = pd.read_csv('seoul_hotel_list.csv', encoding='utf-8-sig')

In [ ]:
df_review.head()

,Unnamed: 0,hotel,star,review
0,0,3001626,NaN,깨끗하게 편히 쉬다 가요~
1,1,3001626,NaN,티비 채널이 많지 않다는 것 빼고는 만족스러웠어요!
2,2,3001626,NaN,너무 편안하게 잘 쉬다 가요!\n숙소도 시청역이나 을지로입구역에서 도보로 갈 수 있...
3,3,3001626,NaN,싸게 잘 잤네요 ㅎㅎ
4,4,3001626,NaN,자주가요 좋습니다


In [ ]:
# 합치기 편하게 column 이름 통일하기
df_review.rename(columns = {'hotel' : 'ID'}, inplace = True)

In [ ]:
df_list.head()

,Unnamed: 0,ID,name
0,1,3000741,써미트 호텔 동대문
1,2,1000112780,어반스테이 더 남산
2,3,3014025,영등포 갤럭시 호텔
3,4,3000607,호텔 더 디자이너스 홍대
4,5,3001137,The-K 호텔 서울


In [ ]:
# ID 열을 기준으로 조인하기
df = pd.merge(df_review, df_list, how='inner', on='ID')

In [ ]:
df.head()

,Unnamed: 0_x,ID,star,review,Unnamed: 0_y,name
0,50,3000741,NaN,만족합니다,1,써미트 호텔 동대문
1,51,3000741,NaN,프론트 직원분들 응대도 친절하시고 역이랑도 멀지 않아서 편했어요! 룸컨디션도 깔끔하...,1,써미트 호텔 동대문
2,52,3000741,NaN,좋습니다.,1,써미트 호텔 동대문
3,53,3000741,NaN,가성비좋아요! 장충체육관에서 가까워서 좋음ㅎ,1,써미트 호텔 동대문
4,54,3000741,NaN,킹성비...굿...,1,써미트 호텔 동대문


In [ ]:
# 필요하지 않은 열 제거하기
df.drop(['Unnamed: 0_x', 'star', 'Unnamed: 0_y'], axis=1)

,ID,review,name
0,3000741,만족합니다,써미트 호텔 동대문
1,3000741,프론트 직원분들 응대도 친절하시고 역이랑도 멀지 않아서 편했어요! 룸컨디션도 깔끔하...,써미트 호텔 동대문
2,3000741,좋습니다.,써미트 호텔 동대문
3,3000741,가성비좋아요! 장충체육관에서 가까워서 좋음ㅎ,써미트 호텔 동대문
4,3000741,킹성비...굿...,써미트 호텔 동대문
...,...,...,...
197834,3000582,객실 깨끗하고 라운지 조식 괜찮았어요\n해피아워 푸드와 술도 괜찮았구요\n수영장은 ...,그랜드 앰배서더 서울 풀만
197835,3000582,모든게좋았으나\n체크인하고방에들어갔는데\n휴지가치워지지않고 뭉텅이로\nTV앞에 있어...,그랜드 앰배서더 서울 풀만
197836,3000582,고급스러운 앰배서더 풀만 아주 좋았습니다.\n결혼식 때문인지 연말 레스토랑 이용객...,그랜드 앰배서더 서울 풀만
197837,3000582,체크인 때 사람이 몰려 좀 오래 기다렸습니다. 미리 예약하고 갔지만 체크인 할 때도...,그랜드 앰배서더 서울 풀만


In [ ]:
# 열 순서 재배치하기
df = df[['ID', 'name', 'review']]

In [ ]:
df.head()

,ID,name,review
0,3000741,써미트 호텔 동대문,만족합니다
1,3000741,써미트 호텔 동대문,프론트 직원분들 응대도 친절하시고 역이랑도 멀지 않아서 편했어요! 룸컨디션도 깔끔하...
2,3000741,써미트 호텔 동대문,좋습니다.
3,3000741,써미트 호텔 동대문,가성비좋아요! 장충체육관에서 가까워서 좋음ㅎ
4,3000741,써미트 호텔 동대문,킹성비...굿...


In [ ]:
# 결측치 확인 : 결측치 없고, 총 197839개의 리뷰가 있음
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 197839 entries, 0 to 197838
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   ID      197839 non-null  int64 
 1   name    197839 non-null  object
 2   review  197839 non-null  object
dtypes: int64(1), object(2)
memory usage: 6.0+ MB


In [ ]:
# 각 전처리 후, 18글자 이하의 리뷰 갯수 비율 확인하는 함수 
def del_percent():
    under_18 = df.loc[df['length'] <= 18,'review'].count()
    all_count = df['review'].count()
    print('18글자 이하 리뷰갯수:', under_18)
    print(under_18 / all_count * 100)

In [ ]:
# 리뷰의 length 측정해서 새로운 열 추가
df['length'] = df['review'].apply(lambda x: len(str(x)))
df['review'] = df['review'].astype('str')

In [ ]:
df.head()

,ID,name,review,length
0,3000741,써미트 호텔 동대문,만족합니다,5
1,3000741,써미트 호텔 동대문,프론트 직원분들 응대도 친절하시고 역이랑도 멀지 않아서 편했어요! 룸컨디션도 깔끔하...,68
2,3000741,써미트 호텔 동대문,좋습니다.,5
3,3000741,써미트 호텔 동대문,가성비좋아요! 장충체육관에서 가까워서 좋음ㅎ,24
4,3000741,써미트 호텔 동대문,킹성비...굿...,10


In [ ]:
# \n(개행문자) 제거
#1.'\n' -> ''
df['review'] = df['review'].apply(lambda x: x.replace('\n',''))
df['length'] = df['review'].apply(lambda x: len(x))
del_percent()

18글자 이하 리뷰갯수: 100702
50.90098514448618


In [ ]:
# 특수문자 제거
import re

# 2. ~, !, ., >
def cleanText(readData):
 
    text = re.sub('[-=+,#/\?:;^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》❤]', '', readData)
    return text

df['review'] = df['review'].apply(cleanText)
df['length'] = df['review'].apply(lambda x: len(x))
del_percent()

18글자 이하 리뷰갯수: 102596
51.85832924751945


In [ ]:
# 의미 없는 모음, 자음 처리하기
def cleanText2(readData):
 
    text = re.sub('[ㄱㄴㄷㄹㅁㅂㅅㅇㅈㅊㅋㅌㅍㅎㅃㅉㄸㄲㅆㅛㅕㅑㅐㅔㅗㅓㅏㅣㅜㅠㅡ]', '', readData)
    return text
df['review'] = df['review'].apply(cleanText2)
df['length'] = df['review'].apply(lambda x: len(x))
del_percent()

18글자 이하 리뷰갯수: 102982
52.05343739100986


In [ ]:
# 이모티콘 제거하기
def cleanText3(readData):
    
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

    text = emoji_pattern.sub(r'', readData)
    return text

df['review'] = df['review'].apply(cleanText3)
df['length'] = df['review'].apply(lambda x: len(x))
del_percent()

18글자 이하 리뷰갯수: 103012
52.06860123635886


# 글자수 전처리
- 500자 이상 제거 (네이버 맞춤법 검사 라이브러리에 돌리기 위함)

In [ ]:
df = df[(20 <= df['length']) & (df['length']<= 500)]
df.shape

(91646, 4)

# 맞춤법 검사 돌리기

In [ ]:
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-uasl64_l
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-uasl64_l
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=3f280542327b2ada90f9e298915be3f2b435e2ea94e24b2ae006785b436ddb1e
  Stored in directory: /tmp/pip-ephem-wheel-cache-ni0c0y4_/wheels/ab/f5/7b/d4124bb329c905301baed80e2ae45aa14e824f62ebc3ec2cc4
Successfully built py-hanspell


In [ ]:
from tqdm import tqdm
from hanspell import spell_checker
from hanspell.constants import CheckResult

In [ ]:
def spell_check(review):
    result = spell_checker.check(review)
    return result.checked

In [ ]:
df['review'] = df['review'].astype('str')

In [ ]:
df.head()

,ID,name,review,length
1,3000741,써미트 호텔 동대문,프론트 직원분들 응대도 친절하시고 역이랑도 멀지 않아서 편했어요 룸컨디션도 깔끔하고...,67
3,3000741,써미트 호텔 동대문,가성비좋아요 장충체육관에서 가까워서 좋음,22
5,3000741,써미트 호텔 동대문,직원분들도 너무 친절하시고 셋이서 침대 하나씩 가격도 시설도 만족했어요 역이랑도 가...,94
10,3000741,써미트 호텔 동대문,침실쪽 내부는 시설 좋았어요 화장실은 쫌 그럭저럭 바닥이 좀 지저분했어요,40
12,3000741,써미트 호텔 동대문,깨끗하고 직원분들도 친절하세요바로근처에 마트편의점도 있어서 재이용했습니다,40


In [ ]:
copy_df = df.copy()
copy_df

,ID,name,review,length
1,3000741,써미트 호텔 동대문,프론트 직원분들 응대도 친절하시고 역이랑도 멀지 않아서 편했어요 룸컨디션도 깔끔하고...,67
3,3000741,써미트 호텔 동대문,가성비좋아요 장충체육관에서 가까워서 좋음,22
5,3000741,써미트 호텔 동대문,직원분들도 너무 친절하시고 셋이서 침대 하나씩 가격도 시설도 만족했어요 역이랑도 가...,94
10,3000741,써미트 호텔 동대문,침실쪽 내부는 시설 좋았어요 화장실은 쫌 그럭저럭 바닥이 좀 지저분했어요,40
12,3000741,써미트 호텔 동대문,깨끗하고 직원분들도 친절하세요바로근처에 마트편의점도 있어서 재이용했습니다,40
...,...,...,...,...
197834,3000582,그랜드 앰배서더 서울 풀만,객실 깨끗하고 라운지 조식 괜찮았어요해피아워 푸드와 술도 괜찮았구요수영장은 실망이었...,78
197835,3000582,그랜드 앰배서더 서울 풀만,모든게좋았으나체크인하고방에들어갔는데휴지가치워지지않고 뭉텅이로TV앞에 있어서기분이않좋...,81
197836,3000582,그랜드 앰배서더 서울 풀만,고급스러운 앰배서더 풀만 아주 좋았습니다결혼식 때문인지 연말 레스토랑 이용객 때문...,374
197837,3000582,그랜드 앰배서더 서울 풀만,체크인 때 사람이 몰려 좀 오래 기다렸습니다 미리 예약하고 갔지만 체크인 할 때도 ...,252


# Sampling
리뷰수가 200개 이상인 호텔을 대상으로 호텔별 리뷰 200개씩 랜덤 추출

In [ ]:
copy_df = copy_df[~copy_df['review'].str.contains("안녕하세요", na=False, case=False)]

In [ ]:
copy_df = copy_df[~copy_df['review'].str.contains("고객님", na=False, case=False)]

In [ ]:
hotel_name = []
for h in copy_df['name']:
    if h not in hotel_name:
        hotel_name.append(h)

In [ ]:
hotel_dic = {}
for h in hotel_name:
  hotel_dic[h] = len(copy_df[copy_df['name'] == h])

hotel_dic # 호텔명 : 리뷰 수

{'DH 네상스 성신여대': 87,
 'ENA 스위트 호텔 남대문': 252,
 'Ever 8 신촌': 77,
 'H Avenue 건대점': 329,
 'H Avenue 성신여대점': 168,
 'H Avenue 이대점': 237,
 'JK 블라썸 호텔': 830,
 'L7 강남 바이 롯데': 787,
 'L7 명동 바이 롯데': 509,
 'SR 호텔 사당점': 531,
 'Staz 스타즈 호텔 명동 1호점': 287,
 'The-K 호텔 서울': 171,
 'WD호텔': 248,
 '강남아르누보씨티 호텔': 277,
 '강남패밀리호텔': 115,
 '골드리버 호텔': 438,
 '구로 JS 호텔': 110,
 '그랜드 머큐어 앰배서더 호텔 앤 레지던스 서울 용산': 172,
 '그랜드 앰배서더 서울 풀만': 164,
 '그랜드 워커힐 서울': 87,
 '그랜드 인터컨티넨탈 서울 파르나스': 131,
 '그랜드 하얏트 서울': 148,
 '글래드 마포': 1066,
 '나비 호텔레지던스': 122,
 '나이아가라 호텔': 330,
 '나인트리 프리미어 명동2': 2288,
 '나인트리 호텔 명동': 449,
 '노보텔 스위트 앰배서더 서울 용산': 443,
 '노보텔 앰배서더 동대문 호텔 & 레지던스': 749,
 '더 리버사이드 호텔': 269,
 '더 리센츠 동대문 호텔': 480,
 '더 파크호텔 서울': 974,
 '도미니크 호텔': 74,
 '도미인 서울 강남': 781,
 '도봉 더데이 호텔': 55,
 '도봉 호텔베이204': 315,
 '동서울 관광호텔': 162,
 '디 아티스트 호텔 성신여대점': 156,
 '디어스명동 (충무로) 호텔': 134,
 '뜨랑블루 호텔': 271,
 '라까사 호텔 서울': 144,
 '라마다 서울 동대문': 761,
 '라마다 앙코르 마곡': 510,
 '라마다 호텔앤스위트 서울 남대문': 496,
 '라미드 호텔': 1284,
 '라이프스타일 F 호텔': 182,
 '라이프스타일 S 호텔': 274,
 '라코

In [ ]:
# 리뷰 수가 200개가 넘는 호텔들만 추출
list200 = []

for i in range(40):
  if list(hotel_dic.values())[i] >= 200:
    list200.append(list(hotel_dic.items())[i])
list200

[('호텔 더 디자이너스 홍대', 338),
 ('롯데시티호텔 명동', 940),
 ('로사나 부티크 호텔', 427),
 ('뜨랑블루 호텔', 271),
 ('이태원 크라운 호텔', 422),
 ('목시 바이 메리어트 서울 인사동', 381),
 ('오라카이 청계산 호텔', 428),
 ('홀리데이인 익스프레스 서울 홍대', 1028),
 ('페어필드 바이 메리어트 서울', 959),
 ('골드리버 호텔', 438),
 ('도미인 서울 강남', 781),
 ('아만티 호텔 서울 홍대', 509),
 ('명동 티마크 그랜드 호텔', 539),
 ('신라스테이 광화문', 905),
 ('이비스 스타일 앰배서더 서울 명동', 207),
 ('어반플레이스 강남', 626),
 ('서울가든호텔', 588),
 ('호텔 피제이 명동', 1027),
 ('스탠포드 호텔 서울', 426),
 ('라마다 호텔앤스위트 서울 남대문', 496),
 ('에이든 바이 베스트웨스턴 청담', 487),
 ('노보텔 앰배서더 동대문 호텔 & 레지던스', 749)]

In [ ]:
# 호텔별 리뷰 200개 random 추출
import random

final_df = pd.DataFrame() # 호텔별 150개의 리뷰를 담을 dataframe 생성

for i in range(len(list200)):
  if i == 0: # 형태소 분석 결과, 빈도가 제대로 추출되지 않은 호텔만 다시 랜덤 샘플링
    random_df = copy_df[copy_df['name'] == list200[i][0]].sample(n=200, random_state=1024) # random_state : 난수 고정
  elif i == 5: # 형태소 분석 결과, 빈도가 제대로 추출되지 않은 호텔만 다시 랜덤 샘플링
    random_df = copy_df[copy_df['name'] == list200[i][0]].sample(n=200, random_state=900) # random_state : 난수 고정
  elif i == 6: # 형태소 분석 결과, 빈도가 제대로 추출되지 않은 호텔만 다시 랜덤 샘플링
    random_df = copy_df[copy_df['name'] == list200[i][0]].sample(n=200, random_state=555) # random_state : 난수 고정
  else:
    random_df = copy_df[copy_df['name'] == list200[i][0]].sample(n=200, random_state=1000) # random_state : 난수 고정
  random_df = random_df.sort_index() # index 정렬 -> 날짜순(최근->과거) 정렬
  final_df = final_df.append(random_df, ignore_index=True)

In [ ]:
final_df[final_df['name'] == '호텔 더 디자이너스 홍대']

,ID,name,review,length
0,3000607,호텔 더 디자이너스 홍대,스텐다드더블이용 사진에 보이는 창문앞 욕조가있는 방을 원했는데 랜덤배정되어 일반 ...,130
1,3000607,호텔 더 디자이너스 홍대,3성급 스위트라고 하기에는 욕실이 좁고 수건이 부족하네요 또 시설이 노후화돼서 여기...,65
2,3000607,호텔 더 디자이너스 홍대,화장실 배수구 물이안내려가는거빼면 너무좋아요 뷰도좋구요,30
3,3000607,호텔 더 디자이너스 홍대,절대 사진으로보고 파티룸 선택하시면 후회하실거예요노래방설치로인해 지하에다 방을 만든...,280
4,3000607,호텔 더 디자이너스 홍대,가격대비 나쁘지않았어요방음이 좀 아쉽고 샤워부스가 없어 뭏이 막 튀어서불편한점 객실...,113
...,...,...,...,...
195,3000607,호텔 더 디자이너스 홍대,우선 홍대입구역과 합정역 사이에 위치해서 접근성이 굉장히 좋습니다 그래서 비싼가봐요...,272
196,3000607,호텔 더 디자이너스 홍대,접근성 좋고 발렛파킹도 되어서 편합니다객실에 충전시설은 없구요청소상태는 솔직히 별로...,120
197,3000607,호텔 더 디자이너스 홍대,접근성과 들어갔을 때 디자인은 좋았습니다 하지만 손을 씻을 때 세면대가 제대로 벽과...,362
198,3000607,호텔 더 디자이너스 홍대,밤9시 객실 청소한다고 우당탕탕호텔이전으로 객실가구 빼는줄복도에서는 청소하시는분들인...,87


In [ ]:
final_df = final_df.reset_index()

In [ ]:
final_df

,index,ID,name,review,length
0,0,3000607,호텔 더 디자이너스 홍대,스텐다드더블이용 사진에 보이는 창문앞 욕조가있는 방을 원했는데 랜덤배정되어 일반 ...,130
1,1,3000607,호텔 더 디자이너스 홍대,3성급 스위트라고 하기에는 욕실이 좁고 수건이 부족하네요 또 시설이 노후화돼서 여기...,65
2,2,3000607,호텔 더 디자이너스 홍대,화장실 배수구 물이안내려가는거빼면 너무좋아요 뷰도좋구요,30
3,3,3000607,호텔 더 디자이너스 홍대,절대 사진으로보고 파티룸 선택하시면 후회하실거예요노래방설치로인해 지하에다 방을 만든...,280
4,4,3000607,호텔 더 디자이너스 홍대,가격대비 나쁘지않았어요방음이 좀 아쉽고 샤워부스가 없어 뭏이 막 튀어서불편한점 객실...,113
...,...,...,...,...,...
4395,4395,3015823,노보텔 앰배서더 동대문 호텔 & 레지던스,정말 휴가차 쉬러 갔었는데 너무나 좋았어요 역시 특급호텔 5스타 저희는 레지던스룸오...,452
4396,4396,3015823,노보텔 앰배서더 동대문 호텔 & 레지던스,기대했던 것 보다 좋았어요이 가격에 아주 잘 쉬고 왔습니다 그래서 재방문의사 강력합...,486
4397,4397,3015823,노보텔 앰배서더 동대문 호텔 & 레지던스,좋은곳이니 어메니티제품도 좋은거였으면 좋겠어요청소상태는 깨끗했는데 먼지가 많이휘날렸...,197
4398,4398,3015823,노보텔 앰배서더 동대문 호텔 & 레지던스,디럭스레지던스 에서 2박 했습니다일단 룸전체가 넓고 화장실도 넓고 다 좋았습니다방에...,113


In [ ]:
final_df = final_df.drop(['index'], axis=1)

In [ ]:
final_df

,ID,name,review,length
0,3000607,호텔 더 디자이너스 홍대,스텐다드더블이용 사진에 보이는 창문앞 욕조가있는 방을 원했는데 랜덤배정되어 일반 ...,130
1,3000607,호텔 더 디자이너스 홍대,3성급 스위트라고 하기에는 욕실이 좁고 수건이 부족하네요 또 시설이 노후화돼서 여기...,65
2,3000607,호텔 더 디자이너스 홍대,화장실 배수구 물이안내려가는거빼면 너무좋아요 뷰도좋구요,30
3,3000607,호텔 더 디자이너스 홍대,절대 사진으로보고 파티룸 선택하시면 후회하실거예요노래방설치로인해 지하에다 방을 만든...,280
4,3000607,호텔 더 디자이너스 홍대,가격대비 나쁘지않았어요방음이 좀 아쉽고 샤워부스가 없어 뭏이 막 튀어서불편한점 객실...,113
...,...,...,...,...
4395,3015823,노보텔 앰배서더 동대문 호텔 & 레지던스,정말 휴가차 쉬러 갔었는데 너무나 좋았어요 역시 특급호텔 5스타 저희는 레지던스룸오...,452
4396,3015823,노보텔 앰배서더 동대문 호텔 & 레지던스,기대했던 것 보다 좋았어요이 가격에 아주 잘 쉬고 왔습니다 그래서 재방문의사 강력합...,486
4397,3015823,노보텔 앰배서더 동대문 호텔 & 레지던스,좋은곳이니 어메니티제품도 좋은거였으면 좋겠어요청소상태는 깨끗했는데 먼지가 많이휘날렸...,197
4398,3015823,노보텔 앰배서더 동대문 호텔 & 레지던스,디럭스레지던스 에서 2박 했습니다일단 룸전체가 넓고 화장실도 넓고 다 좋았습니다방에...,113


In [ ]:
# 특수문자 제거 완료된 리뷰 불러오기
clean_review_col = final_df['review'].copy()
clean_review_col

0       스텐다드더블이용  사진에 보이는 창문앞 욕조가있는 방을 원했는데 랜덤배정되어 일반 ...
1       3성급 스위트라고 하기에는 욕실이 좁고 수건이 부족하네요 또 시설이 노후화돼서 여기...
2                          화장실 배수구 물이안내려가는거빼면 너무좋아요 뷰도좋구요
3       절대 사진으로보고 파티룸 선택하시면 후회하실거예요노래방설치로인해 지하에다 방을 만든...
4       가격대비 나쁘지않았어요방음이 좀 아쉽고 샤워부스가 없어 뭏이 막 튀어서불편한점 객실...
                              ...                        
4395    정말 휴가차 쉬러 갔었는데 너무나 좋았어요 역시 특급호텔 5스타 저희는 레지던스룸오...
4396    기대했던 것 보다 좋았어요이 가격에 아주 잘 쉬고 왔습니다 그래서 재방문의사 강력합...
4397    좋은곳이니 어메니티제품도 좋은거였으면 좋겠어요청소상태는 깨끗했는데 먼지가 많이휘날렸...
4398    디럭스레지던스 에서 2박 했습니다일단 룸전체가 넓고 화장실도 넓고 다 좋았습니다방에...
4399    호텔3시 체크인인데 2시간이나 대기하다가 체크인하고 그리고 직원이 제대로 키나 룸번...
Name: review, Length: 4400, dtype: object

In [ ]:
import re

# 자음/모음 제거 함수 정의
def clean_text(text):
  hangul = re.sub('[ㄱㄴㄷㄹㅁㅂㅅㅇㅈㅊㅋㅌㅍㅎㅃㅉㄸㄲㅆㅛㅕㅑㅐㅔㅗㅓㅏㅣㅜㅠㅡ]', '', text) # 정규표현식에서 자음, 모음 제거
  return hangul

In [ ]:
# 이모티콘 제거 함수 정의
def remove_emoticon(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    hangul = emoji_pattern.sub(r'', text)
    return hangul

In [ ]:
# 새로운 리스트 생성
clean_review = []

# for loop으로 위의 두 함수를 적용한 객체들만 clean_review 리스트에 추가
for i in range(len(clean_review_col)):
  temp_1 = clean_text(clean_review_col[i])
  temp_2 = remove_emoticon(temp_1)
  clean_review.append(temp_2)

In [ ]:
# 기존의 clean_review 컬럼을 전처리(자음/모음 제거, 이모티콘 제거)된 것으로 교체
final_df = final_df.assign(clean_review = [r for r in clean_review])

In [ ]:
# 맞춤법 검사 함수 적용한 preprocessed_review 컬럼 생성
final_df['preprocessed_review'] = final_df['review'].apply(spell_check)

In [ ]:
final_df = final_df[['name', 'review', 'preprocessed_review']] # 필요한 컬럼만 추출
final_df

,name,review,preprocessed_review
0,호텔 더 디자이너스 홍대,스텐다드더블이용 사진에 보이는 창문앞 욕조가있는 방을 원했는데 랜덤배정되어 일반 ...,스탠더드 더블 이용 사진에 보이는 창문 앞 욕조가 있는 방을 원했는데 랜덤 배정되...
1,호텔 더 디자이너스 홍대,3성급 스위트라고 하기에는 욕실이 좁고 수건이 부족하네요 또 시설이 노후화돼서 여기...,3성급 스위트라고 하기에는 욕실이 좁고 수건이 부족하네요 또 시설이 노후화돼서 여기...
2,호텔 더 디자이너스 홍대,화장실 배수구 물이안내려가는거빼면 너무좋아요 뷰도좋구요,화장실 배수구 물이 안 내려가는 거 빼면 너무 좋아요 뷰도 좋고요
3,호텔 더 디자이너스 홍대,절대 사진으로보고 파티룸 선택하시면 후회하실거예요노래방설치로인해 지하에다 방을 만든...,절대 사진으로 보고 파티룸 선택하시면 후회하실 거예요 노래방 설치로 인해 지하에다 ...
4,호텔 더 디자이너스 홍대,가격대비 나쁘지않았어요방음이 좀 아쉽고 샤워부스가 없어 뭏이 막 튀어서불편한점 객실...,가격 대비 나쁘지 않았어요 방음이 좀 아쉽고 샤워부스가 없어 무이 막 튀어서 불편한...
...,...,...,...
4395,노보텔 앰배서더 동대문 호텔 & 레지던스,정말 휴가차 쉬러 갔었는데 너무나 좋았어요 역시 특급호텔 5스타 저희는 레지던스룸오...,정말 휴가차 쉬러 갔었는데 너무나 좋았어요 역시 특급호텔 5스타 저희는 레지던스 룸...
4396,노보텔 앰배서더 동대문 호텔 & 레지던스,기대했던 것 보다 좋았어요이 가격에 아주 잘 쉬고 왔습니다 그래서 재방문의사 강력합...,기대했던 것보다 좋았어요 이 가격에 아주 잘 쉬고 왔습니다 그래서 재방문의 사 강력...
4397,노보텔 앰배서더 동대문 호텔 & 레지던스,좋은곳이니 어메니티제품도 좋은거였으면 좋겠어요청소상태는 깨끗했는데 먼지가 많이휘날렸...,좋은 곳이니 어메니티 제품도 좋은 거였으면 좋겠어요 청소상태는 깨끗했는데 먼지가 많...
4398,노보텔 앰배서더 동대문 호텔 & 레지던스,디럭스레지던스 에서 2박 했습니다일단 룸전체가 넓고 화장실도 넓고 다 좋았습니다방에...,디럭스 레지던스에서 2박 했습니다 일단 룸 전체가 넓고 화장실도 넓고 다 좋았습니다...


# 형태소 분석을 위한 전처리

In [ ]:
# 가성비 -> 구성비 맞춤법 검사기 오류 + 뷰 -> 경치 : 형태소 분석을 위한 처리
cost = ['구성비', '가성비', '가성 비', '가 성비', '가성', '가격 대비']

for i in range(len(final_df)):
  for j in range(len(cost)):
    if cost[j] in final_df['preprocessed_review'][i]:
      final_df['preprocessed_review'][i] = final_df['preprocessed_review'][i].replace(cost[j], " 구성비 ")
  if '뷰' in final_df['preprocessed_review'][i]:
      final_df['preprocessed_review'][i] = final_df['preprocessed_review'][i].replace("뷰", "경치")
  if '더 보기' in final_df['preprocessed_review'][i]:
      final_df['preprocessed_review'][i] = final_df['preprocessed_review'][i].replace("더 보기", "")

In [ ]:
final_df.to_csv('preprocessed.csv', encoding='utf-8-sig')

In [ ]:
final_df.to_excel('preprocessed.xlsx', encoding='utf-8-sig')